In [ ]:
!pip install --upgrade gdown
import gdown

# https://drive.google.com/file/d/1_z3rAKRlNjLHlp2Ry_RHkctXDOK-YVIm/view?usp=sharing
file_id = "1_z3rAKRlNjLHlp2Ry_RHkctXDOK-YVIm"
gdown.download(f"https://drive.google.com/uc?id={file_id}", output="buka.ba-20250403T160643Z-001.zip", quiet=False)


Downloading...
From (original): https://drive.google.com/uc?id=1_z3rAKRlNjLHlp2Ry_RHkctXDOK-YVIm
From (redirected): https://drive.google.com/uc?id=1_z3rAKRlNjLHlp2Ry_RHkctXDOK-YVIm&confirm=t&uuid=9426ffd4-00b0-4a85-94c7-b1ced309cfbc
To: /content/buka.ba-20250403T160643Z-001.zip
100%|██████████| 233M/233M [00:03<00:00, 65.1MB/s]


'buka.ba-20250403T160643Z-001.zip'

In [ ]:
import zipfile

with zipfile.ZipFile("buka.ba.zip", 'r') as zip_ref:
    zip_ref.extractall("buka.ba")


In [ ]:
import os
import re
import json
import csv
import glob
from pathlib import Path

def extract_articles_from_file(file_path):
    """Extract individual articles from a text file."""
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    # Split by "<***>" marker
    articles = content.split("<***>")

    # Remove empty entries
    articles = [article.strip() for article in articles if article.strip()]

    return articles

def parse_article(article_text):
    """Parse article text to extract structured data."""
    article_data = {
        "Url": "",
        "Date": "",
        "Title": "",
        "Text": ""
    }

    # Extract URL
    link_match = re.search(r'LINK:\s*(https?://[^\s]+)', article_text)
    if link_match:
        article_data["Url"] = link_match.group(1)

    # Extract Date
    date_match = re.search(r'DATUM:\s*(\d{2}\.\d{2}\.\d{4})', article_text)
    if date_match:
        article_data["Date"] = date_match.group(1)

    # Extract Title - Fixed to properly extract what comes after "NASLOV:"
    title_match = re.search(r'NASLOV:\s*([^\n]+)', article_text)
    if title_match:
        article_data["Title"] = title_match.group(1).strip()

    # Extract Text (everything after the metadata section)
    text_parts = article_text.split("LINK:")
    if len(text_parts) > 1:
        # Find the first occurrence of two consecutive newlines after LINK
        text_content = text_parts[1].split("\n\n", 1)
        if len(text_content) > 1:
            article_data["Text"] = text_content[1].strip()

    return article_data

def process_txt_files(base_folder):
    """Process all txt files in all subdirectories of the base folder."""
    all_articles = []

    # Get all txt files in all subdirectories
    txt_files = glob.glob(os.path.join(base_folder, "**/*.txt"), recursive=True)

    for txt_file in txt_files:
        print(f"Processing file: {txt_file}")

        # Extract directory path for output files
        dir_path = os.path.dirname(txt_file)
        filename = os.path.basename(txt_file).split('.')[0]

        # Extract articles from the file
        articles = extract_articles_from_file(txt_file)

        # Parse each article
        parsed_articles = []
        for article in articles:
            article_data = parse_article(article)
            if any(article_data.values()):  # Only add if we extracted something
                parsed_articles.append(article_data)

        # Skip if no articles were parsed
        if not parsed_articles:
            print(f"No articles found in {txt_file}")
            continue

        # Save to JSON
        json_file = os.path.join(dir_path, f"{filename}.json")
        with open(json_file, 'w', encoding='utf-8') as f:
            json.dump(parsed_articles, f, ensure_ascii=False, indent=4)

        # Save to CSV
        csv_file = os.path.join(dir_path, f"{filename}.csv")
        with open(csv_file, 'w', encoding='utf-8', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=["Url", "Date", "Title", "Text"])
            writer.writeheader()
            writer.writerows(parsed_articles)

        print(f"Created {json_file} and {csv_file}")
        all_articles.extend(parsed_articles)

    return all_articles

def main():
    # Base folder path - the inner buka.ba folder
    base_folder = "/content/buka.ba/buka.ba"

    # Check if the folder exists
    if not os.path.exists(base_folder):
        print(f"Error: Folder {base_folder} does not exist.")
        return

    # Process all txt files
    articles = process_txt_files(base_folder)

    # Print summary
    print(f"\nProcessing complete! Extracted {len(articles)} articles from all files.")

if __name__ == "__main__":
    main()

Processing file: /content/buka.ba/buka.ba/Intervju/IntervjuSviClanci.txt
Created /content/buka.ba/buka.ba/Intervju/IntervjuSviClanci.json and /content/buka.ba/buka.ba/Intervju/IntervjuSviClanci.csv
Processing file: /content/buka.ba/buka.ba/Region/RegionSviClanciPrviDio.txt
Created /content/buka.ba/buka.ba/Region/RegionSviClanciPrviDio.json and /content/buka.ba/buka.ba/Region/RegionSviClanciPrviDio.csv
Processing file: /content/buka.ba/buka.ba/Region/RegionSviClanciDrugiDio.txt
Created /content/buka.ba/buka.ba/Region/RegionSviClanciDrugiDio.json and /content/buka.ba/buka.ba/Region/RegionSviClanciDrugiDio.csv
Processing file: /content/buka.ba/buka.ba/Region/RegionSviClanciTreciDio.txt
Created /content/buka.ba/buka.ba/Region/RegionSviClanciTreciDio.json and /content/buka.ba/buka.ba/Region/RegionSviClanciTreciDio.csv
Processing file: /content/buka.ba/buka.ba/BiH/BiHSviClanciDrugiDio.txt
Created /content/buka.ba/buka.ba/BiH/BiHSviClanciDrugiDio.json and /content/buka.ba/buka.ba/BiH/BiHSviCl

In [ ]:
import os
import json
import csv
import glob
from pathlib import Path

def extract_articles_from_file(file_path):
    """Extract individual articles from a text file."""
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    # Split by "<***>" marker
    articles = content.split("<***>")

    # Remove empty entries
    articles = [article.strip() for article in articles if article.strip()]

    return articles

def parse_article(article_text):
    """Parse article text to extract structured data using line-by-line approach."""
    article_data = {
        "Url": "",
        "Date": "",
        "Title": "",
        "Text": ""
    }

    # Split the article into lines for processing
    lines = article_text.split('\n')
    metadata_section = True
    text_lines = []

    # Dictionary to collect all metadata lines
    metadata = {}

    for line in lines:
        line = line.strip()

        # Skip empty lines
        if not line:
            continue

        # Check if we're still in the metadata section
        if metadata_section:
            # Process metadata lines
            if ":" in line:
                # Split on first colon
                parts = line.split(":", 1)
                if len(parts) > 1:
                    key = parts[0].strip()
                    value = parts[1].strip()
                    metadata[key] = value

                    # Check if this line contains "LINK" which signifies end of metadata
                    if "LINK" in key:
                        metadata_section = False
        else:
            # We're in the text section now
            text_lines.append(line)

    # Now process collected metadata to extract our fields
    if "DATUM" in metadata:
        article_data["Date"] = metadata["DATUM"]

    if "NASLOV" in metadata:
        article_data["Title"] = metadata["NASLOV"]
        print(f"Found title: '{metadata['NASLOV']}'")

    if "LINK" in metadata:
        article_data["Url"] = metadata["LINK"]

    # Join all text lines
    if text_lines:
        article_data["Text"] = ' '.join(text_lines)

    # Print final data for debugging
    print(f"Final extracted data:")
    print(f"Title: {article_data['Title']}")
    print(f"Date: {article_data['Date']}")
    print(f"URL: {article_data['Url']}")
    print(f"Text length: {len(article_data['Text'])} characters")

    return article_data

def process_txt_files(base_folder):
    """Process all txt files in all subdirectories of the base folder."""
    all_articles = []

    # Get all txt files in all subdirectories
    txt_files = glob.glob(os.path.join(base_folder, "**/*.txt"), recursive=True)

    for txt_file in txt_files:
        print(f"Processing file: {txt_file}")

        # Extract directory path for output files
        dir_path = os.path.dirname(txt_file)
        filename = os.path.basename(txt_file).split('.')[0]

        # Extract articles from the file
        articles = extract_articles_from_file(txt_file)

        # Parse each article
        parsed_articles = []
        for i, article in enumerate(articles):
            print(f"\nProcessing article {i+1} in {txt_file}")

            article_data = parse_article(article)
            if any(article_data.values()):  # Only add if we extracted something
                parsed_articles.append(article_data)

        # Skip if no articles were parsed
        if not parsed_articles:
            print(f"No articles found in {txt_file}")
            continue

        # Save to JSON
        json_file = os.path.join(dir_path, f"{filename}.json")
        with open(json_file, 'w', encoding='utf-8') as f:
            json.dump(parsed_articles, f, ensure_ascii=False, indent=4)

        # Save to CSV
        csv_file = os.path.join(dir_path, f"{filename}.csv")
        with open(csv_file, 'w', encoding='utf-8', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=["Url", "Date", "Title", "Text"])
            writer.writeheader()
            writer.writerows(parsed_articles)

        print(f"Created {json_file} and {csv_file}")
        all_articles.extend(parsed_articles)

    return all_articles

def main():
    # Base folder path - the inner buka.ba folder
    base_folder = "/content/buka.ba/buka.ba"

    # Check if the folder exists
    if not os.path.exists(base_folder):
        print(f"Error: Folder {base_folder} does not exist.")
        return

    # Process all txt files
    articles = process_txt_files(base_folder)

    # Print summary
    print(f"\nProcessing complete! Extracted {len(articles)} articles from all files.")

if __name__ == "__main__":
    main()

Streaming output truncated to the last 5000 lines.
Date: 20.05.2015
URL: https://6yka.com/ekonomija/kakve-nas-cijene-roaminga-ocekuju-od-1-juna/
Text length: 843 characters

Processing article 6354 in /content/buka.ba/buka.ba/Ekonomija/EkonomijaSviClanciPrviDio.txt
Found title: 'Javna preduzeća u dubiozu nisu potonula sama, njima je upravljala politika'
Final extracted data:
Title: Javna preduzeća u dubiozu nisu potonula sama, njima je upravljala politika
Date: 07.02.2024
URL: https://6yka.com/ekonomija/javna-preduzeca-u-dubiozu-nisu-potonula-sama-njima-je-upravljala-politika/
Text length: 7475 characters

Processing article 6355 in /content/buka.ba/buka.ba/Ekonomija/EkonomijaSviClanciPrviDio.txt
Found title: 'Novi udar na džepove, najavljuju se poskupljenja u kafanama..'
Final extracted data:
Title: Novi udar na džepove, najavljuju se poskupljenja u kafanama..
Date: 22.12.2021
URL: https://6yka.com/ekonomija/novi-udar-na-dzepove-najavljuju-se-poskupljenja-u-kafanama/
Text length: 844 

In [ ]:
# TO download it to my PC
import shutil
shutil.make_archive('/content/buka.ba', 'zip', '/content/buka.ba')

'/content/buka.ba.zip'

In [ ]:
from google.colab import files
files.download('/content/buka.ba.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>